In [1]:
from transformers import AutoTokenizer, OPTForCausalLM

model_name = "facebook/opt-1.3b"
model = OPTForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "What is the meaning of life?"
inputs = tokenizer(prompt, return_tensors="pt")
# Generated logits.
logits_huggingface = model.forward(inputs.input_ids, inputs.attention_mask, return_dict=False)
print(logits_huggingface[0])

/home/sungsoon/repos/SHARK/shark.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[[-1.0043, -1.2243,  4.6494,  ..., -0.8709, -1.0086, -0.9346],
         [-6.0413, -5.9169, -2.9258,  ..., -6.1264, -5.8013, -6.0853],
         [-5.8734, -5.9453, -2.6071,  ..., -5.9422, -5.6971, -5.6782],
         ...,
         [-6.0484, -6.2182, -2.1765,  ..., -6.2328, -6.1445, -6.1472],
         [-4.3969, -3.8920,  7.8763,  ..., -4.3238, -4.7918, -4.3455],
         [-3.6522, -4.1917,  8.0274,  ..., -3.6928, -3.6756, -3.6851]]],
       grad_fn=<UnsafeViewBackward0>)


In [ ]:
from shark.shark_inference import SharkInference
from transformers import AutoTokenizer
import os


model_name = "facebook/opt-1.3b"
vmfb_path = 'opt-1.3b_causallm_30_torch_cpu-sync.vmfb'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
shark_module = SharkInference(mlir_module=None, device='cpu-sync')
shark_module.load_module(vmfb_path)

prompt = "What is the meaning of life?"
model_inputs = tokenizer(
    prompt,
    padding="max_length",
    max_length=30,
    truncation=True,
    return_tensors="pt"
)
inputs = (
    model_inputs['input_ids'],
    model_inputs['attention_mask'],
)
# Generate logits output of OPT model.
logits = shark_module('forward', inputs)
# Print output logits to validate vs. pytorch + base transformers
print(logits[0])


In [3]:
l0= logits_huggingface[0]

In [4]:
type(l0)

torch.Tensor

In [5]:
l0.shape

torch.Size([1, 8, 50272])

In [6]:
l1 = logits_huggingface[0][0]

In [7]:
type(l1)

torch.Tensor

In [8]:
l1.shape

torch.Size([8, 50272])

In [10]:
dir(l1)

['H',
 'T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr_

In [21]:
l1.tolist()

[[-1.0042625665664673,
  -1.2243491411209106,
  4.649369716644287,
  -0.9896318316459656,
  2.8047361373901367,
  -2.2967045307159424,
  1.751700520515442,
  -1.6462355852127075,
  0.9280180931091309,
  0.2846640646457672,
  -1.7984375953674316,
  -2.014465570449829,
  7.496925354003906,
  -1.3645368814468384,
  -0.26818394660949707,
  -1.6182905435562134,
  0.4260113835334778,
  9.01863956451416,
  2.970065116882324,
  -1.6966993808746338,
  1.8911758661270142,
  0.32813242077827454,
  1.4182690382003784,
  -1.7817713022232056,
  -0.796669065952301,
  -0.8702408075332642,
  0.8202935457229614,
  1.3600714206695557,
  -1.3586934804916382,
  6.149946689605713,
  -1.9712445735931396,
  -1.256489634513855,
  0.20209816098213196,
  0.282164990901947,
  -0.13931545615196228,
  4.468037128448486,
  1.2156360149383545,
  -0.9869641065597534,
  2.379621744155884,
  -2.083298444747925,
  -0.9132774472236633,
  -2.681905746459961,
  -1.2664668560028076,
  3.4756369590759277,
  0.7151868343353271

In [12]:
l2

<function Tensor.values>

In [24]:
type(logits_huggingface[0])

torch.Tensor